# Scripts del proyecto MLOps



Script 1: Preparacion de datos para el entrenamiento

In [ ]:
import os
import zipfile
from typing import Tuple

import pandas as pd
import numpy as np
import wget
from sklearn.model_selection import train_test_split


def download_data(data_path: str) -> None:
    """Descarga los datos necesarios para el procesamiento desde una URL y los guarda en una carpeta local.

    Args:
        data_path (str): Ruta de la carpeta donde se guardarán los datos descargados.
    """
    train_link = 'https://github.com/kubeflow/examples/blob/master/digit-recognition-kaggle-competition/data/train.csv.zip?raw=true'
    wget.download(train_link, os.path.join(data_path, 'data_csv.zip'))

    with zipfile.ZipFile(os.path.join(data_path, 'data_csv.zip'), "r") as zip_ref:
        zip_ref.extractall(data_path)


def read_file_csv(filename: str, data_path: str) -> pd.DataFrame:
    """Lee un archivo CSV y devuelve un DataFrame.

    Args:
        filename (str): Nombre del archivo CSV.
        data_path (str): Ruta de la carpeta donde se encuentra el archivo CSV.

    Returns:
        pd.DataFrame: DataFrame creado a partir del archivo CSV.
    """
    train_data_path = os.path.join(data_path, filename)
    df = pd.read_csv(train_data_path)
    print(filename, 'cargado correctamente')
    return df


def data_process(dataframe: pd.DataFrame) -> None:
    """Procesa los datos, dividiéndolos en conjuntos de entrenamiento, prueba y validación, y los guarda en archivos NPZ.

    Args:
        dataframe (pd.DataFrame): DataFrame que contiene los datos a procesar.
    """
    ntrain = dataframe.shape[0]

    all_data_X = dataframe.drop('label', axis=1)
    all_data_y = dataframe.label

    all_data_X = all_data_X.values.reshape(-1, 28, 28, 1)
    all_data_X = all_data_X / 255.0

    X = all_data_X[:ntrain].copy()
    y = all_data_y[:ntrain].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

    np.savez(os.path.join('../data/processed/', 'train_data.npz'), X=X_train, y=y_train)
    print('Data de entrenamiento exportada correctamente en la carpeta processed')

    np.savez(os.path.join('../data/processed/', 'test_data.npz'), X=X_test, y=y_test)
    print('Data de prueba exportada correctamente en la carpeta processed')

    np.savez(os.path.join('../data/processed/', 'val_data.npz'), X=X_val, y=y_val)
    print('Data de validacion exportada correctamente en la carpeta processed')


def main() -> None:
    """Función principal para la preparación de datos."""
    data_path = "../data/raw"
    download_data(data_path)
    df1 = read_file_csv('train.csv', data_path)
    data_process(df1)


if __name__ == "__main__":
    main()



Script 2: Entrenamiento del modelo

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics
from typing import Tuple


def load_data(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """Carga los datos de entrenamiento desde un archivo NPZ.

    Args:
        filename (str): Nombre del archivo NPZ que contiene los datos.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Tupla con los datos de entrada (X) y las etiquetas (y).
    """
    data_path = os.path.join('../data/processed', filename)
    train_data = np.load(data_path)
    X_train, y_train = train_data['X'], train_data['y']
    return X_train, y_train


def build_model(hidden_dim1: int, hidden_dim2: int, dropout_rate: float = 0.5) -> models.Sequential:
    """Construye y retorna un modelo secuencial de Keras.

    Args:
        hidden_dim1 (int): Número de filtros para la primera capa convolucional.
        hidden_dim2 (int): Número de filtros para la segunda capa convolucional.
        dropout_rate (float, optional): Tasa de dropout para las capas de dropout. Por defecto es 0.5.

    Returns:
        models.Sequential: Modelo secuencial construido.
    """
    model = models.Sequential([
        layers.Conv2D(filters=hidden_dim1, kernel_size=(5, 5), padding='same', activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Conv2D(filters=hidden_dim2, kernel_size=(3, 3), padding='same', activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Conv2D(filters=hidden_dim2, kernel_size=(3, 3), padding='same', activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Flatten(),
        layers.Dense(10, activation="softmax")
    ])

    model.build(input_shape=(None, 28, 28, 1))

    return model


def train_model(model: models.Sequential, X_train: np.ndarray, y_train: np.ndarray,
                learning_rate: float, epochs: int, batch_size: int) -> tf.keras.callbacks.History:
    """Configura y entrena el modelo.

    Args:
        model (models.Sequential): Modelo de Keras a entrenar.
        X_train (np.ndarray): Datos de entrada de entrenamiento.
        y_train (np.ndarray): Etiquetas de entrenamiento.
        learning_rate (float): Tasa de aprendizaje para el optimizador.
        epochs (int): Número de épocas de entrenamiento.
        batch_size (int): Tamaño del lote para el entrenamiento.

    Returns:
        tf.keras.callbacks.History: Historia del entrenamiento del modelo.
    """
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
                  loss=losses.SparseCategoricalCrossentropy(),
                  metrics=[metrics.SparseCategoricalAccuracy(name='accuracy')])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

    # Guardar el modelo
    model_save_path = 'models/best_model.h5'
    if not os.path.isdir('models'):
        os.makedirs('models')
    model.save(model_save_path)
    print(f'Modelo guardado en {model_save_path}')

    print('El entrenamiento del modelo ha finalizado con éxito.')

    return history


def main() -> None:
    """Función principal para el entrenamiento del modelo."""
    # Hiperparámetros
    LR = 1e-3
    EPOCHS = 2
    BATCH_SIZE = 64
    CONV_DIM1 = 56
    CONV_DIM2 = 100

    X_train, y_train = load_data('train_data.npz')
    model = build_model(CONV_DIM1, CONV_DIM2)
    history = train_model(model, X_train, y_train, LR, EPOCHS, BATCH_SIZE)

    print('El entrenamiento del modelo ha finalizado con éxito.')


if __name__ == "__main__":
    main()


Script 3: Evaluacion del modelo

In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import os
from tensorflow.keras.models import load_model
from typing import Tuple


def load_data(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """Carga los datos de entrenamiento desde un archivo NPZ.

    Args:
        filename (str): Nombre del archivo NPZ que contiene los datos.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Tupla con los datos de entrada (X) y las etiquetas (y).
    """
    data_path = os.path.join("../data/processed", filename)
    train_data = np.load(data_path)
    X_train, y_train = train_data["X"], train_data["y"]
    return X_train, y_train


def eval_model(X_test: np.ndarray, y_test: np.ndarray) -> None:
    """Evalúa el modelo cargado con los datos de prueba y muestra la matriz de confusión.

    Args:
        X_test (np.ndarray): Datos de entrada de prueba.
        y_test (np.ndarray): Etiquetas de prueba.

    Returns:
        None
    """
    # Ruta al modelo guardado
    model_path = "models/best_model.h5"

    # Cargar el modelo
    model = load_model(model_path)

    print("Modelo cargado exitosamente")

    # Evaluación del modelo
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test_loss: {test_loss}, Test_accuracy: {test_acc}")

    # Predicciones del modelo
    y_pred = np.argmax(model.predict(X_test), axis=-1)

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)

    # Visualización de la matriz de confusión
    plt.figure(figsize=(7, 7))
    sns.heatmap(cm, fmt="g", cbar=False, annot=True, cmap="Blues")
    plt.title("Matriz de confusión")
    plt.ylabel("Etiqueta real")
    plt.xlabel("Etiqueta predicha")
    plt.show()


def main() -> None:
    """Función principal para la evaluación del modelo."""
    X_test, y_test = load_data("test_data.npz")

    eval_model(X_test, y_test)
    print("Finalizó la evaluación del modelo")


if __name__ == "__main__":
    main()


Script 4: Scoring del modelo

In [ ]:
import os
from typing import Tuple

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model


def load_data(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """Carga los datos de entrenamiento desde un archivo NPZ.

    Args:
        filename (str): Nombre del archivo NPZ.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Datos de entrada y etiquetas cargados.
    """
    data_path = os.path.join('../data/processed', filename)
    train_data = np.load(data_path)
    X_train, y_train = train_data['X'], train_data['y']
    return X_train, y_train


def score_model(X_val: np.ndarray, y_val: np.ndarray) -> None:
    """Evalúa el modelo cargado y guarda los resultados en un archivo CSV.

    Args:
        X_val (np.ndarray): Datos de entrada de validación.
        y_val (np.ndarray): Etiquetas de validación.
    """
    # Ruta al modelo guardado
    model_path = 'models/best_model.h5'

    # Cargar el modelo
    model = load_model(model_path)
    print("Modelo cargado exitosamente")

    y_pred = np.argmax(model.predict(X_val), axis=-1)

    results_df = pd.DataFrame({
        'Real': y_val,
        'Predicted': y_pred
    })
    results_df.to_csv(os.path.join('../data/scores/score.csv'),index=False)
    print('Exportación correctamente en la carpeta scores')


def main() -> None:
    """Función principal para la evaluación del modelo."""
    X_test, y_test = load_data('val_data.npz')
    score_model(X_test, y_test)
    print('Finalizó la validación del Modelo')


if __name__ == "__main__":
    main()
